## Exercise 4: External Data Integration & Business Recommendations

In [134]:
import pandas as pd
import json
from sqlalchemy import create_engine
import time
from meteostat import Point, Daily
from geopy.geocoders import Nominatim


##### 0. Informe

**Estrategia**

Como compañía de seguros, el objetivo es tener la mayor cantidad de pólizas al mejor precio posible mientras minimizamos la cantidad de reclamaciones. Debido a esto, podemos analizar dos factores: 

- cómo de probable es que un cliente realice alguna acción que provoque una reclamación
- qué tan probable es que un factor externo cause un problema que derive en una reclamación

Ya que inferir resultados basados en condiciones personales podría ser difícil o disriminatorio, vamos a enfocarnos en factores externos.

**Dataset elegido**

Los datos con los que estamos trabajando continene información variada de cada cliente (ciudad, profesión, edad) pero la cantidad de datos es limitada. Un análisis interesante puede ser agrupar a los clientes por ciudad aunque resultaría en un análisis muy genérico. Por eso, podemos considerar agregar datos externos como los datos meteorológicos históricos de cada una de las ciudades en las que los clientes han realizado una reclamación. 

Además, el clima afecta a todas las pólizas ofrecidas (salud, automóviles, hogar...) por lo que parece un buen punto de partida.

**Cómo ayudaría a la empresa**

- Predecir qué clientes son más propensos a realizar reclamaciones en función del clima de su zona.
- Ayuda a ajustar las pólizas en regiones con condiciones climáticas de riesgo.
- Envío de alertas personalizadas a clientes según el histórico del clima, por ejemplo, "se acerca temporada de muchaslluvias, revisa el estado de tu tejado".
- Ofrecer productos adicionales según el clima

##### 1. Connect to APIs

Let's collect the weather data 7 days prior to every claim in the corresponding city.

We will use geopy for location data and meteostat for weather data.

Geopy has a free API, just needing registration in https://www.geonames.org/

Meteostat has a free API with no login required.

In [136]:
# connection to geopy
geolocator = Nominatim(user_agent="vjm1996")


##### 2. Load data from DB

In [36]:

with open("../config.json", "r") as f:
    config = json.load(f)

    db_name = config["DB_NAME"]
    db_user = config["DB_USER"]
    db_password = config["DB_PASSWORD"]
    db_host = config["DB_HOST"]
    db_port = int(config["DB_PORT"])

DB_URL = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
engine = create_engine(DB_URL)

customers = pd.read_sql("SELECT * FROM customers", con=engine)
policies = pd.read_sql("SELECT * FROM policies", con=engine)
claims = pd.read_sql("SELECT * FROM claims", con=engine)
risk_indicators = pd.read_sql("SELECT * FROM risk_indicators", con=engine)
print("Data read from DB!")

Connecting to PostgreSQL...
Data read from DB!


##### 3. Collecting data

First, finding out which cities and dates to collect

In [137]:
# Removing those with missing date (2020-01-01 by default)
claims_filtered = claims[claims['claim_date'] != '2020-01-01']

# Left claims with customer cities
claims_merged_cities = pd.merge(claims_filtered, customers[['customer_id','state']], on='customer_id', how='left')

# Removing duplicate data for city and date
unique_data = claims_merged_cities[['claim_date','state']].drop_duplicates()

Then querying APIs to collect data

In [159]:
# Since APIs probably have rate limits, it is better to use a for loop
# Otherwise, a concurrent approach could be chosen.

# For a small loop, all weather will be stored in a dataframe. For a bigger loop, data could be stored directly in DB
all_weather = pd.DataFrame({})

for index, row in unique_data.iterrows():

    print("Finding weather data for " + row['state'] + " -> " + str(row['claim_date']))
    location = geolocator.geocode(row['state'] + " Spain")

    # Set time period (one week to also collect incidents during a week)
    start = row['claim_date'] - pd.Timedelta(days=7)
    end = row['claim_date']

    # Create Point for city
    city = Point(location.latitude, location.longitude)

    # Get daily data - weather for each day
    data = Daily(city, start, end)
    data = data.fetch()

    # Add city and date to weather data
    data['state'] = row['state']
    data.reset_index(inplace=True)
    
    # Adding data to general table
    all_weather = pd.concat([all_weather, data], ignore_index=True)

    # Sleeping to not overload API
    time.sleep(0.5)

    # Exiting after 10 cities to not overload API - just for testing
    if index >= 10:
        break

Finding weather data for león -> 2020-07-31 00:00:00
Finding weather data for león -> 2023-01-04 00:00:00
Finding weather data for león -> 2022-12-26 00:00:00
Finding weather data for huesca -> 2022-05-10 00:00:00
Finding weather data for vizcaya -> 2024-05-18 00:00:00
Finding weather data for vizcaya -> 2022-05-01 00:00:00
Finding weather data for vizcaya -> 2022-06-25 00:00:00
Finding weather data for vizcaya -> 2022-01-18 00:00:00
Finding weather data for vizcaya -> 2021-12-29 00:00:00
Finding weather data for vizcaya -> 2022-05-07 00:00:00
Finding weather data for vizcaya -> 2024-08-02 00:00:00


#### 4. Cleaning data

In [164]:
all_weather.isnull().sum()

time      0
tavg      0
tmin      0
tmax      0
prcp      0
snow     88
wdir      0
wspd      0
wpgt      0
pres      0
tsun     88
state     0
dtype: int64

In [ ]:
all_weather.drop_duplicates(inplace=True)
all_weather.fillna(all_weather.mean(numeric_only=True), inplace=True)
cleaned_df = all_weather.drop(columns=['snow', 'tsun'])
cleaned_df.isnull().sum()

#### 5. Storing data in DB

In [167]:
cleaned_df.to_sql("historical_weather", con=engine, if_exists="append", index=False)

88

#### 6. Merging weather data to claims df (only specific day)

In [75]:
claims_with_weather = pd.merge(claims_merged_cities,all_weather,left_on=["claim_date","state"],right_on=["time","state"])

Now analysis could continue, investigating if most claims happen during bad weather or any other hypotheses.